# SQL Alchemy: Drill

We will create a small database that contains quotes from authors and the name of the author. For this exercise, we will use `SQLite` which is very easy to implement. But don't forget that the advantage of ORMs is that they can be used in the same way, no matter what the database.

## What Is SQLite?  

[SQLite](https://www.sqlite.org/index.html) is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. SQLite is the most used database engine in the world. It is built into all mobile phones and most computers and comes bundled inside countless other applications that people use every day.

The imports: <p style="color:red;"><b >⚠️ You must have SQL_Alchemy version 1.4 or higher</b></p>

We have already imported some helpful modules to approach the drill. Don't forget to check the documentation to review some of them as needed. 

In [1]:
#!pip install sqlalchemy==1.4

In [5]:
from sqlalchemy import create_engine
import datetime as dt
from sqlalchemy import Column, Date, Integer, Text, create_engine, inspect
from sqlalchemy.orm import registry, sessionmaker

## ORM configuration
---

**1. Create an `engine` variable that contains an instance of `create_engine()`.**  
 
You must specify the type of database, in this case sqlite, and specify the path to the database `/content/data.db`. *(You can put the path you want if you don't use Google Colab)* `create_engine()` will create the data.db file if it does not exist.

Creating an engine in SQLAlchemy is a fundamental step for interacting with a database. Here’s why you would create an engine and what it enables you to do:

### Purpose of Creating an Engine

**1. Establish a Database Connection:**

The engine acts as the primary interface for connecting to your database. It provides a source of connectivity that SQLAlchemy uses to execute SQL commands and transactions.

**2. Database Operations:**

Once the engine is created, you can perform various database operations, such as creating tables, inserting data, querying data, and managing transactions.

**3. Configuration Management:**

The engine allows you to configure and manage database-specific settings like connection pooling, timeout settings, and echoing SQL statements for debugging.

**4. SQL Execution:**

The engine enables raw SQL execution through methods like `engine.execute()`, which is useful for running custom SQL commands directly.

**5. Session Management:**

The engine is used by SQLAlchemy’s ORM (Object-Relational Mapping) system to create sessions, which are used to manage the database transactions and operations within your application.


In [6]:
engine = create_engine('sqlite:///NewDataBase.db')

**2. Create a `mapper_registry` variable which contains an instance of `registry()`.**

In SQLAlchemy, `registry` is a part of the SQLAlchemy 2.0 API and represents a new approach to managing mappings between Python classes and database tables. This approach is more explicit and flexible compared to the older `declarative_base()` method. Here’s a detailed explanation:

### What is `registry`?

`registry` is a function from the `sqlalchemy.orm` module that creates a new `MapperRegistry` object. This object manages the mapping between your Python classes and the database tables. It provides an alternative to the traditional `declarative_base()` approach used for ORM (Object-Relational Mapping) in SQLAlchemy.


In [7]:
mapper_registry = registry()

**3. Create a `Base` variable which contains an instance of `mapper_registry.generate_base()`.**  

This method allows to manage and create the models that we will use later.

This line of code creates a new base class for your ORM (Object-Relational Mapping) models. The base class is a crucial part of SQLAlchemy’s ORM system because it is used to define and manage the mappings between your Python classes and database tables.

### Purpose of the Base Class:

**1. Central Point for Mappings:**
- The base class serves as a central point for defining and managing your ORM models. All of your model classes (which represent database tables) will inherit from this base class.

**2. Metadata Management:**
- The base class holds the metadata associated with the ORM models. Metadata includes information about the table structure, columns, and relationships.


In [8]:
Base = mapper_registry.generate_base()

**4. Create a `Maker` variable that contains an instance of `sessionmaker()`.**  

This class allows us to manage transactions with the database.

In [10]:
Maker = sessionmaker(bind=engine)

### `sessionmaker` Function:

- **`sessionmaker`** is a factory function provided by SQLAlchemy for creating `Session` objects. A `Session` is a workspace for interacting with the database, where you perform transactions and queries.

### `bind=engine`:

- The `bind` parameter connects the `Session` to the specific `engine` (which is created using `create_engine`). The `engine` defines the database connection parameters, such as the database type and location.
- By binding the `engine` to the `sessionmaker`, you ensure that all sessions created by this factory will use the same database connection configuration.

### `Maker` Variable:

- The `Maker` variable now holds a session factory. It is not yet an actual session, but a callable that can be used to create `Session` instances.


**5. Create a `session` variable.**  

This variable contains an instance of `Maker` with the parameter `bind` which has the value `engine`.

In [11]:
session = Maker()

## Creation of the model
---

**6. Create a `class Quotation` which inherits from the previously created `Base` class.**

This class will have 4 attributes: 

- **\_\_tablename\_\_**: which contains the name of the `quotation` table that we are going to use.

- **id**: which has the method `Column`. This column will be an `Integer` attribute and will have the `primary_key` attribute set to `True`

- **quote_quotation**: which has the method `Column` which will have an attribute `Text` and the attribute `nullable` set to `False`

- **quote_author**: which has the `Column` method and will have a `Text` attribute and the `nullable` attribute set to `False`



In [12]:
class Quotation(Base):
    __tablename__ = "quotation"
    
    id = Column(Integer, primary_key=True)
    quote_quotation = Column(Text, nullable=False)
    quote_author = Column(Text, nullable=False)

**7. Create the table using the method `Base.metadata.create_all()`.**  
Assign the `engine` value to the bind `parameter`.

### 1. What `Base.metadata.create_all()` Does:

- **Purpose:**
  - The method `create_all()` is used to create all tables defined by models that inherit from the `Base` class in the specified database.

- **Context:**
  - `Base` is an instance of `declarative_base` (or `MapperRegistry.generate_base()` in SQLAlchemy 2.0) which keeps track of all the ORM-mapped classes and their metadata.

### 2. Metadata Object:

- **Metadata:**
  - `Base.metadata` is an instance of SQLAlchemy's `MetaData` object. This object stores information about the tables and columns defined in the ORM models.

- **Tables Definition:**
  - `Base.metadata` includes a list of all the tables that have been defined by the ORM classes which inherit from `Base`. This list is derived from the `__table__` attribute in each class.

### 3. `engine` Parameter:

- **Engine:**
  - The `engine` is a SQLAlchemy object that represents the database connection. It provides the database dialect and connection details.

- **Bind:**
  - The `engine` parameter is used to execute SQL commands. It connects to the actual database where the tables will be created.

### 4. Internal Execution:

- **Inspection:**
  - SQLAlchemy inspects `Base.metadata` and finds the `quotation` table.

- **SQL Generation:**
  - It generates a `CREATE TABLE` statement for the `quotation` table based on the metadata information.


In [13]:
Base.metadata.create_all(engine)

## The transactions
---

### Create

**8. Create a varible `new_quotation` that contains an instance of `Quotation`.** 

For the `quote_quotation` parameter assign the following sentence:
> “The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”

For the parameter `quote_author` assign the value *'Albert Einstein'*

In [14]:
new_quotation = Quotation(quote_quotation="The world as we created it is a process of our thinking. It cannot be changed without changing our thinking.", quote_author = "Albert Einstein")

**9. You will have to add the transaction. To do this, use the `session.add() ` method with `new_quotation` as argument.**

In [15]:
session.add(new_quotation)

**10. The transaction has been added but not yet executed. To finish the transaction, you must use the `session.commit()` method.**
You don't have to pass any arguments.

In [16]:
session.commit()

**11. Add all these sentences to the database (use a single commit):**

In [17]:
data = [
    {"author": "Oscar Wilde", "text": "Beauty is in the eyes of the beholder."},
    {"author": "Marcel Proust", "text": "Beauty is not in the colors, but in their harmony."},
    {"author": "Philippe Claudel", "text": "Each letter has a scent, each verb, a perfume. Each word diffuses in the memory a place and its fragrances. And the text that gradually weaves itself, with the combined hazards of the alphabet and remembrance, then becomes the wonderful river, a thousand times branched and fragrant, of our dreamed life, our lived life, our life to come, which in turn carries us away and reveals us."},
    {"author": "Tintin", "text": "That’s one small step for man, on giant leap for mankind."},
    {"author": "François Weyergans", "text": "A dream that is not interpreted is like a letter that is not read."},
    {"author": "Vincent Van Gogh", "text": "I dream my painting, then I paint my dream."},
    {"author": "Hugues de Saint-Victor", "text": "An idle young man is like a young bull without the yoke."},
    {"author": "George Bernard Shaw", "text": "Marriage is the story of a young man and a young girl who pick a flower and receive an avalanche on their heads."},
    {"author": "Alain Tastet", "text": "I never dream at night, my only dream is to be awake beside you..."},
    {"author": "Oscar Wilde", "text":"Be yourself; everyone else is already taken."},
    {"author": "Alain Tastet", "text":"This is a false quote..."}
]

In [18]:
for element in data:
    quote = Quotation(quote_quotation=element["text"], quote_author=element["author"])
    session.add(quote)
session.commit()        

### Read

**12. Display all quotes and authors in this form: 'This is a quote' - Author**

### Session Query

- `session.query(Quotation)`: This creates a SQLAlchemy `Query` object for the `Quotation` class. This object represents a query that will be executed against the database to retrieve records from the `quotation` table.

### Retrieve All Records

- `.all()`: This method executes the query and retrieves all records from the `quotation` table. The result is returned as a list of `Quotation` objects.


In [19]:
quotes = session.query(Quotation).all()

for quote in quotes:
    print(f"'{quote.quote_quotation}' - {quote.quote_author}")

'The world as we created it is a process of our thinking. It cannot be changed without changing our thinking.' - Albert Einstein
'Beauty is in the eyes of the beholder.' - Oscar Wilde
'Beauty is not in the colors, but in their harmony.' - Marcel Proust
'Each letter has a scent, each verb, a perfume. Each word diffuses in the memory a place and its fragrances. And the text that gradually weaves itself, with the combined hazards of the alphabet and remembrance, then becomes the wonderful river, a thousand times branched and fragrant, of our dreamed life, our lived life, our life to come, which in turn carries us away and reveals us.' - Philippe Claudel
'That’s one small step for man, on giant leap for mankind.' - Tintin
'A dream that is not interpreted is like a letter that is not read.' - François Weyergans
'I dream my painting, then I paint my dream.' - Vincent Van Gogh
'An idle young man is like a young bull without the yoke.' - Hugues de Saint-Victor
'Marriage is the story of a young

**13. Display the sentence with `id 8`.**

In [22]:
quote = session.query(Quotation).filter_by(id=8).first()
print(f"'{quote.quote_quotation}'")

'An idle young man is like a young bull without the yoke.'


**14. Display all quotes by `Oscar Wilde`.**


In [25]:
quotes = session.query(Quotation).filter_by(quote_author='Oscar Wilde').all()
for quote in quotes:
    print(f"'{quote.quote_quotation}'")

'Beauty is in the eyes of the beholder.'
'Be yourself; everyone else is already taken.'


### Update 

**15. Make an update of the sentence:** 
>"That’s one small step for man, on giant leap for mankind." 

and **replace the author who is `Tintin` by `Neil Armstrong`**.

In [28]:
quote_to_update = session.query(Quotation).filter_by(
    quote_quotation="That’s one small step for man, on giant leap for mankind."
).first()

quote_to_update.quote_author = "Neil Armstrong"
session.commit()

updated_quote = session.query(Quotation).filter_by(quote_quotation="That’s one small step for man, on giant leap for mankind.").first()
print(f"Updated quote: '{updated_quote.quote_quotation}' - {updated_quote.quote_author}")


Updated quote: 'That’s one small step for man, on giant leap for mankind.' - Neil Armstrong


### Delete

**16. Delete all quotes by `Alain Tastet`.**

In [29]:
quotes_to_delete = session.query(Quotation).filter_by(quote_author='Alain Tastet').all()

for quote in quotes_to_delete:
    session.delete(quote)

session.commit()

remaining_quotes = session.query(Quotation).filter_by(quote_author='Alain Tastet').all()
if not remaining_quotes:
    print("No remaining quotes by Alain Tastet.")

No remaining quotes by Alain Tastet.


**17. Delete the quote with the `id 2`.** 

In [30]:
quote_to_delete = session.query(Quotation).filter_by(id=2).first()
    
session.delete(quote_to_delete)

session.commit()

remaining_quotes = session.query(Quotation).filter_by(id=2).first()
if not remaining_quotes:
    print("No remaining quotes with id 2.")

No remaining quotes with id 2.


## That's it!
You have built a database and acquired many new tools to interact with it. On to the next step!

![tools](https://media.giphy.com/media/xT0xekLZmeC54FCTJu/giphy-downsized-large.gif) 

## Resources

- [Udacity Database Setup using SQLAlchemy](https://github.com/udacity/Full-Stack-Foundations/blob/master/Lesson_1/database_setup.py)
- [Cheat sheet SQL Alchemy](https://www.pythonsheets.com/notes/python-sqlalchemy.html)